In [ ]:
%pip install simpy

In [3]:
import simpy
import random
import statistics
from collections import deque
import pandas as pd
import datetime
import numpy as np

In [141]:
# Params and constants. In minutes

#Global Simpy env variables
SEED = 1234
SIM_TIME = 60  # 8 hours of operation #can add a scalar like .5 for testing with a shorter time and to preserve the times in dict below

#Variables for project analysis
#PAX_ARRIVALS = 1/5 # The expo rate passengers arrive at the monorail station.
HEADWAY = 5 # Cyclical minutes between each train's arrival. Constant set by monorail's operators
DWELL_TIME = 3 # Minutes that a train remains in the station to allow for passenger loading. Constant set by monorail's operators
CAR_CAPACITY = 10 # Number of passengers that the park's model of trains car can hold.
CARS_PER_TRAIN = 3 # Number of train cars coupled together

# Schedule to define peak times throughout park open hours, in 30 min intervals
# Assume empiral data collection determined a peak in the morning at opening time, then again around lunch when people leave and come back. It tapers out like a tail to end the day
arrival_schedule = {1: 15, 2: 12, 3: 10, 4: 5, # First two hours, 8AM to 10AM
                    5: 3, 6: 4, 7: 3, 8: 6, # Next two hours, 10AM to 12PM
                    9: 25, 10: 22, 11: 14, 12: 8, # Next two hours, 12PM to 2PM
                    13: 6, 14: 4, 15: 3 , 16: 1# Final two hours, 2PM to4PM
                    }


In [131]:
def passenger_arrivals(env, store):
    """ Simpy function to generate passenger arrivals at the station platform and add to the FIFO queue"""
    pax_counter = 0

    
    while True:
        #Determine which interval the clock is in, 30 min intervals
        interval = (env.now // 30) + 1
        pax_interarrival = random.expovariate(arrival_schedule[interval]*.05) #remove .2, just slowing down for testing
        yield env.timeout(pax_interarrival)
        pax_counter += 1

        store.put(pax_counter)
        
        pax_queue.append(pax_counter)
        pax_id_list.append(pax_counter)
        pax_arrival_times.append(env.now)
        
        
        
        
        mins, seconds = divmod(env.now * 60, 60) #this time format only for printing
        print(f"Passenger {pax_counter} arrived at t={int(mins)}:{int(seconds):02d} mins")
        #print(pax_queue)

        

In [25]:
def train_arrival(env, store, headway=HEADWAY, capacity=CAR_CAPACITY, n_cars=CARS_PER_TRAIN, dwell_time=DWELL_TIME):
    """ Simpy function to create the train arrivals at determined cadence"""
    total_capacity = capacity * n_cars
    train_counter = 0
    
    while True:
        
        yield env.timeout(HEADWAY)
        train_counter += 1

        #Jump into loading process function
        env.process(pax_loading(env, total_capacity, dwell_time, train_counter, store))
        
        mins, seconds = divmod(env.now * 60, 60)  #this time format only for printing
        print(f"=={n_cars} Car monorail with capacity {total_capacity} pax and ID {train_counter} arrived at t={int(mins)}:{int(seconds):02d} mins==")

In [147]:
def pax_loading(env, total_capacity, dwell_time, train_counter, store):
    """ Will remove people from the queue and load the train at a stochastic rate.
    Right now i have a uniform dist for testing values. we might want to change this to something more sophisticated
    """
    # Counter to keep track of pax loaded with each call of this function
    loaded = 0

    # Helpful output
    if len(pax_queue) != 0: #mind the case when a 0 len queue gives an index error
        print(f"ID starting platform queue = {pax_queue[0]}")
    elif len(pax_queue) == 0:
        print(f"Waiting for passengers on platform!")
        
    print(f"==Monorail dwells and loads for max of {dwell_time} mins==")

    #Elapsed times variables
    train_arrival_time = env.now
    train_departure_time = float(env.now + dwell_time)

    # Repeats until dwell reached
    while env.now < train_departure_time:        
        # Condition for max capacity. ensures no more depart system
        if loaded == total_capacity:
            print(f"==Monorail capacity reached! Queue n={len(pax_queue)}. Sitting then leaving station in t={train_departure_time - env.now:.2f} mins==")
            yield env.timeout(train_departure_time - env.now) #Reaching capacity before dwell time, train must sit until set departure time

        # Condition for people up to capacity
        elif loaded < total_capacity:
            load_time = random.triangular(0, .5, .25)
            if env.now + load_time < train_departure_time:

                #Waits for arrival if platform clear, or grabs from queue
                on_platform = store.get()
                #End time
                dwell_end = env.timeout(train_departure_time - env.now)
                
                condition = yield on_platform | dwell_end  

                if on_platform in condition:
                    yield env.timeout(load_time)
                    loaded += 1
                    print(f"Loaded passenger n={loaded} @ {env.now:.2f}")            
                    pax_queue.popleft()
                    
                    pax_departure_times.append(env.now)
                    n_trains_waited.append(train_counter)
                    queue_len_snapshot.append(len(pax_queue))        
    
    print(f"==Dwell reached for train {train_counter}. Loaded {loaded} pax. Monorail departs at t={env.now}==")
    if len(pax_queue) != 0:
        print(f"Queue n={len(pax_queue)}. ID ending platform queue = {pax_queue[0]}")
    elif len(pax_queue) == 0:
        print(f"Platform cleared! Queue n={len(pax_queue)}.")

In [149]:

#Stat collection data structures
pax_id_list = []
pax_arrival_times = []
pax_departure_times = []
n_trains_waited = []
queue_len_snapshot = []

pax_queue = deque() # FIFO queue to manage people arriving at and leaving in order. Stores pax id and arrival timestamp. Eg (id, time) or (4, 1.32)

#random.seed(SEED)
# or
seed = random.seed(datetime.datetime.now().timestamp())
print(seed)

env = simpy.Environment()
pax_store = simpy.Store(env)


#Processes
env.process(passenger_arrivals(env, pax_store))
env.process(train_arrival(env, pax_store))


env.run(until=SIM_TIME)

#Stat collection

None
Passenger 1 arrived at t=0:12 mins
Passenger 2 arrived at t=0:42 mins
Passenger 3 arrived at t=0:45 mins
Passenger 4 arrived at t=1:13 mins
Passenger 5 arrived at t=3:07 mins
Passenger 6 arrived at t=3:12 mins
Passenger 7 arrived at t=3:40 mins
Passenger 8 arrived at t=4:20 mins
Passenger 9 arrived at t=4:24 mins
==3 Car monorail with capacity 30 pax and ID 1 arrived at t=5:00 mins==
ID starting platform queue = 1
==Monorail dwells and loads for max of 3 mins==
Loaded passenger n=1 @ 5.29
Loaded passenger n=2 @ 5.38
Loaded passenger n=3 @ 5.67
Loaded passenger n=4 @ 5.98
Loaded passenger n=5 @ 6.22
Loaded passenger n=6 @ 6.49
Loaded passenger n=7 @ 6.67
Loaded passenger n=8 @ 6.80
Passenger 10 arrived at t=7:03 mins
Loaded passenger n=9 @ 7.14
Loaded passenger n=10 @ 7.29
Passenger 11 arrived at t=7:34 mins
Loaded passenger n=11 @ 7.92
==Dwell reached for train 1. Loaded 11 pax. Monorail departs at t=8.0==
Platform cleared! Queue n=0.
Passenger 12 arrived at t=9:30 mins
==3 Car mo

In [143]:
print(pax_id_list)
print(pax_arrival_times)
print(pax_departure_times)

print(n_trains_waited)

print(len(pax_id_list))
print(len(pax_arrival_times))
print(len(pax_departure_times))
print(len(n_trains_waited))


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]
[1.1221128435520105, 2.4648664346626283, 4.949129318070684, 5.726929065479897, 5.7947757435266, 9.3336289035123, 9.987228982093292, 10.260024452176223, 12.567684199450754, 13.283530331018536, 13.66629410505401, 15.082547089934225, 16.01506473906427, 17.136362271248803, 20.175367891214528, 22.912361780141854, 24.102429419393726, 25.965830531353767, 27.712575002941747, 28.399534771690973, 29.10486596602129, 30.152335576692813, 32.42740830360242, 33.44974436462053, 34.706663980252, 38.94560484982986, 40.01311551308883, 40.42580536632477, 49.58352186178264, 50.50973350634811, 50.64549987176527, 56.10115417421689, 56.568713978151514, 58.87446641854319]
[5.134974521693849, 5.216749929432415, 5.365118372622478, 6.001877070827965, 6.22945503843445, 10.19862719797563, 10.46944385315136, 12.96932856978657, 15.292957539373175, 15.541009528058527, 16.295119944359264, 17.4

In [144]:
print(pax_queue)
print(len(pax_queue)) #Number left waiting at simlation end time
#should match this
print(len(pax_arrival_times)-len(pax_departure_times))

deque([26, 27, 28, 29, 30, 31, 32, 33, 34])
9
9


In [150]:


time_data = {
    'pax_id': pax_id_list,
    'pax_arrival_time': pax_arrival_times,
    'pax_departure_time': pax_departure_times,
    'n_trains_waited': n_trains_waited,
    'queue_len_snapshot': queue_len_snapshot
}
df = pd.DataFrame.from_dict((time_data), orient='index').T
display(df)

,pax_id,pax_arrival_time,pax_departure_time,n_trains_waited,queue_len_snapshot
0,1.0,0.213542,5.290089,1.0,8.0
1,2.0,0.715991,5.381146,1.0,7.0
2,3.0,0.755141,5.665581,1.0,6.0
3,4.0,1.219303,5.981420,1.0,5.0
4,5.0,3.129582,6.224146,1.0,4.0
5,6.0,3.206720,6.489348,1.0,3.0
6,7.0,3.676455,6.670926,1.0,2.0
7,8.0,4.339346,6.804544,1.0,1.0
8,9.0,4.400151,7.136887,1.0,1.0
9,10.0,7.061547,7.286294,1.0,0.0
